In [1]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [2]:
list1=[[1],[2],[3],[4],[5],[6],[7],[8],[9]]
df1=spark.createDataFrame(list1, ['customer_id'])


list2=[[1, '2019-04-08',20],
       [1, '2019-02-04',25],
       [1, '2019-01-04',26],
       [2, '2020-02-04',27],
       [2,'2019-01-08',30],
       [2, '2019-01-03',31],
       [3, '2020-08-01',32],
       [3, '2019-01-04',41],
       [4, '2020-01-08',45],
       [5,'2021-04-01',49],
       [6,'2019-04-01',51],
       [6,'2019-02-01',62]]
df2=spark.createDataFrame(list2, ['customer_id', 'created_at','amount'])

df1.show()
df2.show()

df1.createOrReplaceTempView("customers")
df2.withColumn("created_at", F.to_timestamp("created_at")).createOrReplaceTempView("loans")

+-----------+
customer_id|
+-----------+
 1|
 2|
 3|
 4|
 5|
 6|
 7|
 8|
 9|
+-----------+

+-----------+----------+------+
customer_id|created_at|amount|
+-----------+----------+------+
 1|2019-04-08| 20|
 1|2019-02-04| 25|
 1|2019-01-04| 26|
 2|2020-02-04| 27|
 2|2019-01-08| 30|
 2|2019-01-03| 31|
 3|2020-08-01| 32|
 3|2019-01-04| 41|
 4|2020-01-08| 45|
 5|2021-04-01| 49|
 6|2019-04-01| 51|
 6|2019-02-01| 62|
+-----------+----------+------+

In [3]:
spark.sql("""select a.customer_id from customers a join (select customer_id, count(*) from loans group by customer_id having count(*)>1) b on a.customer_id=b.customer_id""").show()

+-----------+
customer_id|
+-----------+
 6|
 1|
 3|
 2|
+-----------+

In [4]:
spark.sql("""select a.customer_id from customers a join (select customer_id, count(*) from loans group by customer_id having count(*)>1) b on a.customer_id=b.customer_id""").show()

+-----------+
customer_id|
+-----------+
 6|
 1|
 3|
 2|
+-----------+

In [5]:
spark.sql("""select a.customer_id, b.amount from customers a join (select amount, customer_id, year(created_at) as YEAR, (row_number() over (partition by customer_id order by created_at)) as RN from loans) b on a.customer_id=b.customer_id where YEAR=2019 and RN=1 """).show()

+-----------+------+
customer_id|amount|
+-----------+------+
 6| 62|
 1| 26|
 3| 41|
 2| 31|
+-----------+------+

In [6]:
spark.sql("""select * from loans""").show()

+-----------+-------------------+------+
customer_id| created_at|amount|
+-----------+-------------------+------+
 1|2019-04-08 00:00:00| 20|
 1|2019-02-04 00:00:00| 25|
 1|2019-01-04 00:00:00| 26|
 2|2020-02-04 00:00:00| 27|
 2|2019-01-08 00:00:00| 30|
 2|2019-01-03 00:00:00| 31|
 3|2020-08-01 00:00:00| 32|
 3|2019-01-04 00:00:00| 41|
 4|2020-01-08 00:00:00| 45|
 5|2021-04-01 00:00:00| 49|
 6|2019-04-01 00:00:00| 51|
 6|2019-02-01 00:00:00| 62|
+-----------+-------------------+------+

In [7]:
list=[[[1,1,1]],
      [[3,4,5]],
      [[1,2,1,2]]]

df=spark.createDataFrame(list, ["col1"])
df.show()

+------------+
 col1|
+------------+
 [1, 1, 1]|
 [3, 4, 5]|
[1, 2, 1, 2]|
+------------+

In [8]:
df.withColumn("count", F.size(F.array_distinct("col1"))).show()

+------------+-----+
 col1|count|
+------------+-----+
 [1, 1, 1]| 1|
 [3, 4, 5]| 3|
[1, 2, 1, 2]| 2|
+------------+-----+

In [9]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import pandas as pd
Page =   ['H','H','H','H','H','H','H','H','H']
Article = ['A','B','C','A','B','C','D','C','B']
Rank =    [1,1,1,2,2,2,3,3,3]
CountOfRank = [50,30,10,40,30,10,40,30,50]

df = spark.createDataFrame(pd.DataFrame([Page,Article,Rank,CountOfRank]).T, schema=["Group",'Article', 'Rank','CountOfRank'])
df.show()


+-----+-------+----+-----------+
Group|Article|Rank|CountOfRank|
+-----+-------+----+-----------+
 H| A| 1| 50|
 H| B| 1| 30|
 H| C| 1| 10|
 H| A| 2| 40|
 H| B| 2| 30|
 H| C| 2| 10|
 H| D| 3| 40|
 H| C| 3| 30|
 H| B| 3| 50|
+-----+-------+----+-----------+

In [11]:
w=Window().partitionBy("Group","Rank").orderBy(F.col("CountOfRank").desc())
w2=Window().partitionBy("Group","Article","max").orderBy(F.col("Rank").asc())
df.withColumn("max", F.row_number().over(w))\
.withColumn("max1", F.dense_rank().over(w2)).show()








'''.filter(F.col("CountOfRank")==F.col("first"))\
.withColumn("max2", F.row_number().over(w)).filter(F.col("max2")==1)\
.drop("max","first","max2").orderBy("Article").show()

'''

+-----+-------+----+-----------+---+----+
Group|Article|Rank|CountOfRank|max|max1|
+-----+-------+----+-----------+---+----+
 H| B| 1| 30| 2| 1|
 H| B| 2| 30| 2| 2|
 H| A| 1| 50| 1| 1|
 H| A| 2| 40| 1| 2|
 H| D| 3| 40| 2| 1|
 H| B| 3| 50| 1| 1|
 H| C| 1| 10| 3| 1|
 H| C| 2| 10| 3| 2|
 H| C| 3| 30| 3| 3|
+-----+-------+----+-----------+---+----+

Out[3]: '.filter(F.col("CountOfRank")==F.col("first")).withColumn("max2", F.row_number().over(w)).filter(F.col("max2")==1).drop("max","first","max2").orderBy("Article").show()\n\n'

In [12]:
w=Window().partitionBy("Group","Rank").orderBy(F.col("CountOfRank").desc())
w2=Window().partitionBy("Group","Article","max").orderBy("max")
df.withColumn("max", F.row_number().over(w))\
.withColumn("first", F.first("CountOfRank").over(w2))\
.orderBy("Article","max").filter(F.col("CountOfRank")==F.col("first"))\
.withColumn("max2", F.row_number().over(w)).filter(F.col("max2")==1)\
.drop("max","first","max2").orderBy("Article").show()


In [13]:
+-----+-------+----+-----------+
|Group|Article|Rank|CountOfRank|
+-----+-------+----+-----------+
|    H|      A|   1|         50|
|    H|      B|   2|         30|
|    H|      D|   3|         40|
+-----+-------+----+-----------+

In [14]:
+--------+----------+-----------+-------+
|Customer|FlightDate|IssuingDate|Revenue|
+--------+----------+-----------+-------+
|       H|         A|          1|     50|
|       H|         B|          2|     30|
|       H|         D|          3|     40|
+--------+----------+-----------+-------+

In [15]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

list=[['two',0.6,1.2,1.7,1.5,1.4,2.0,1],
      ['one',0.3,1.2,1.3,1.5,1.4,1.0,1],
      ['three',0.4,1.2,1.5,1.9,1.2,0.9,1],
      ['four',0.9,1.2,1.4,1.9,2.5,1.2,1],
      ['five',0.9,1.2,1.4,1.9,2.5,1.2,1]]

df=spark.createDataFrame(list, ['num','col1','col2','col3','col4','col5','col6'])

In [16]:
def transpose(df):

   return  df.withColumn('concat',F.split((F.concat_ws(',',(*(x for x in df.columns[1:])))),','))\
             .drop(*(x for x in df.columns[1:])).groupBy().pivot(df.columns[0]).agg(F.first("concat"))\
             .withColumn("met", F.explode(F.arrays_zip(*(x[0] for x in df.collect()))))\
             .select("met.*")
  
df.show()
transpose(df).show()

+-----+----+----+----+----+----+----+---+
 num|col1|col2|col3|col4|col5|col6| _8|
+-----+----+----+----+----+----+----+---+
 two| 0.6| 1.2| 1.7| 1.5| 1.4| 2.0| 1|
 one| 0.3| 1.2| 1.3| 1.5| 1.4| 1.0| 1|
three| 0.4| 1.2| 1.5| 1.9| 1.2| 0.9| 1|
 four| 0.9| 1.2| 1.4| 1.9| 2.5| 1.2| 1|
 five| 0.9| 1.2| 1.4| 1.9| 2.5| 1.2| 1|
+-----+----+----+----+----+----+----+---+

+---+---+-----+----+----+
two|one|three|four|five|
+---+---+-----+----+----+
0.6|0.3| 0.4| 0.9| 0.9|
1.2|1.2| 1.2| 1.2| 1.2|
1.7|1.3| 1.5| 1.4| 1.4|
1.5|1.5| 1.9| 1.9| 1.9|
1.4|1.4| 1.2| 2.5| 2.5|
2.0|1.0| 0.9| 1.2| 1.2|
 1| 1| 1| 1| 1|
+---+---+-----+----+----+

In [17]:
def transpose(df):
           df1= df.withColumn('concat',F.split((F.concat_ws(',',(*(x for x in df.columns[1:])))),','))\
                 .drop(*(x for x in df.columns[1:])).groupBy().pivot(df.columns[0]).agg(F.first("concat"))
           return df1.withColumn("met", F.explode(F.arrays_zip(*(df1.columns))))\
                 .select("met.*")
df.show()
transpose(df).show()

+-----+----+----+----+----+----+----+---+
 num|col1|col2|col3|col4|col5|col6| _8|
+-----+----+----+----+----+----+----+---+
 two| 0.6| 1.2| 1.7| 1.5| 1.4| 2.0| 1|
 one| 0.3| 1.2| 1.3| 1.5| 1.4| 1.0| 1|
three| 0.4| 1.2| 1.5| 1.9| 1.2| 0.9| 1|
 four| 0.9| 1.2| 1.4| 1.9| 2.5| 1.2| 1|
 five| 0.9| 1.2| 1.4| 1.9| 2.5| 1.2| 1|
+-----+----+----+----+----+----+----+---+

+----+----+---+-----+---+
five|four|one|three|two|
+----+----+---+-----+---+
 0.9| 0.9|0.3| 0.4|0.6|
 1.2| 1.2|1.2| 1.2|1.2|
 1.4| 1.4|1.3| 1.5|1.7|
 1.9| 1.9|1.5| 1.9|1.5|
 2.5| 2.5|1.4| 1.2|1.4|
 1.2| 1.2|1.0| 0.9|2.0|
 1| 1| 1| 1| 1|
+----+----+---+-----+---+

In [18]:
from pyspark.sql import functions as F
list=[['24-MAY-2019']]


df=spark.createDataFrame(list, ['date'])

In [19]:
df.show()

+-----------+
 date|
+-----------+
24-MAY-2019|
+-----------+

In [20]:
df.withColumn("date1", F.to_timestamp("date",'dd-MMM-yyyy')).show()

+-----------+-------------------+
 date| date1|
+-----------+-------------------+
24-MAY-2019|2019-05-24 00:00:00|
+-----------+-------------------+

In [21]:
from pyspark.sql import functions as F
list=[[1,'0','1','0','2','tre',100],
     [2,'1','0','0','0','tre',200],
     [3,'2','0','1','1','abc',300]]

df=spark.createDataFrame(list, ['id','pt_1','pt_2','pt_3','pt_5','name','amt'])
df.show()

df.select((*(x for x in df.columns if not x.startswith('pt'))),*(F.col(x).cast("long").alias(x) for x in df.columns if x.startswith('pt'))).withColumn("pt", F.greatest(*[x for x in df.columns if x.startswith('pt')])).show()




+---+----+----+----+----+----+---+
 id|pt_1|pt_2|pt_3|pt_5|name|amt|
+---+----+----+----+----+----+---+
 1| 0| 1| 0| 2| tre|100|
 2| 1| 0| 0| 0| tre|200|
 3| 2| 0| 1| 1| abc|300|
+---+----+----+----+----+----+---+

+---+----+---+----+----+----+----+---+
 id|name|amt|pt_1|pt_2|pt_3|pt_5| pt|
+---+----+---+----+----+----+----+---+
 1| tre|100| 0| 1| 0| 2| 2|
 2| tre|200| 1| 0| 0| 0| 1|
 3| abc|300| 2| 0| 1| 1| 2|
+---+----+---+----+----+----+----+---+